In [1]:
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [2]:
import pandas as pd
from pathlib import Path
import os
root = Path("data/preprocessed/persuasion")
# Example path template
files = root.rglob("*/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)

In [6]:
df.head(3)

,filename,start,end,text_lang,text,label_bin,labels_multi,source_file
0,pl_abortion_11_04_2024_n01.txt,0,266,PL,Wicemarszałek Włodzimierz Czarzasty:\nWznawiam...,False,[],data/preprocessed/persuasion/trail/trail-pl/ba...
1,pl_abortion_11_04_2024_n01.txt,268,371,PL,"Panowie, halo, szanujmy wszystkich gości.\nDzi...",False,[],data/preprocessed/persuasion/trail/trail-pl/ba...
2,pl_abortion_11_04_2024_n01.txt,373,911,PL,"Przystępujemy do rozpatrzenia punktów 10.,\n11...",False,[],data/preprocessed/persuasion/trail/trail-pl/ba...


In [3]:
df['source_file'].value_counts().sort_index()

source_file
data/preprocessed/persuasion/trail/trail-bg/batch_0000.parquet    20
data/preprocessed/persuasion/trail/trail-bg/batch_0001.parquet    20
data/preprocessed/persuasion/trail/trail-bg/batch_0002.parquet    20
data/preprocessed/persuasion/trail/trail-bg/batch_0003.parquet    15
data/preprocessed/persuasion/trail/trail-pl/batch_0000.parquet    20
data/preprocessed/persuasion/trail/trail-pl/batch_0001.parquet     7
data/preprocessed/persuasion/trail/trail-ru/batch_0000.parquet    18
data/preprocessed/persuasion/trail/trail-si/batch_0000.parquet     9
data/preprocessed/persuasion/train/train-bg/batch_0000.parquet    20
data/preprocessed/persuasion/train/train-bg/batch_0001.parquet    20
data/preprocessed/persuasion/train/train-bg/batch_0002.parquet    20
data/preprocessed/persuasion/train/train-bg/batch_0003.parquet    20
data/preprocessed/persuasion/train/train-bg/batch_0004.parquet    20
data/preprocessed/persuasion/train/train-bg/batch_0005.parquet    20
data/preprocessed/pers

In [12]:
# Explode the multi-label column
exploded = df[["text_lang", "labels_multi"]].explode("labels_multi")

# Create pivot table: count of each label per language
pivot = exploded.pivot_table(
    index="labels_multi",
    columns="text_lang",
    aggfunc="size",  # or 'count'
    fill_value=0
)

# Optional: sort by index or columns
pivot = pivot.sort_index()
pivot

text_lang,BG,PL,RU,SI
labels_multi,,,,
Appeal_to_Authority,7,20,8,29
Appeal_to_Fear-Prejudice,20,31,25,25
Appeal_to_Hypocrisy,29,23,8,16
Appeal_to_Pity,2,34,4,9
Appeal_to_Popularity,12,29,3,20
Appeal_to_Time,2,19,6,2
Appeal_to_Values,8,78,24,34
Causal_Oversimplification,17,15,21,8
Consequential_Oversimplification,14,6,12,11


In [4]:
from collections import Counter

# Flatten all labels_multi lists into one list
all_labels = df["labels_multi"].explode()

# Drop None or NaN values (in case some entries have empty label lists)
all_labels = all_labels.dropna()

# Count occurrences
label_counts = Counter(all_labels)

# Convert to DataFrame for display or saving
label_counts_df = pd.DataFrame(
    label_counts.items(), columns=["label", "count"]
).sort_values("count", ascending=False)

print(label_counts_df)

                               label  count
13                   Loaded_Language    184
3                              Doubt    147
15        Questioning_the_Reputation    146
2                   Appeal_to_Values    144
9              Name_Calling-Labeling    108
5                         Repetition    106
12          Appeal_to_Fear-Prejudice    101
6          Exaggeration-Minimisation    101
20               Appeal_to_Hypocrisy     76
0                Conversation_Killer     72
4               Appeal_to_Popularity     64
24               Appeal_to_Authority     64
17         Causal_Oversimplification     61
23                       Red_Herring     51
1                     Appeal_to_Pity     49
22                       Flag_Waving     45
19  Consequential_Oversimplification     43
8    Obfuscation-Vagueness-Confusion     39
10           False_Dilemma-No_Choice     33
16                         Straw_Man     31
18                           Slogans     31
21                      Whatabou